#### Checking Python version

In [1]:
!python -V

Python 3.8.8


This is the same code present in `persistency_base_model - modified data2.ipynb`. I have just copied the notebook and renamed it

#### Importing libraries

In [2]:
import os
import warnings
warnings.simplefilter("ignore", UserWarning)

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import datetime as dt

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
%matplotlib inline

from feature_engine import encoding as ce
from feature_engine import imputation as mdi
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
import sklearn.metrics as metrics

#### Importing mlflow and setting tracking uri

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("persistency-prediction-experiment")

<Experiment: artifact_location='./mlruns/1', creation_time=1673260171550, experiment_id='1', last_update_time=1673260171550, lifecycle_stage='active', name='persistency-prediction-experiment', tags={}>

#### Importing data

In [4]:
INPUT_FILEPATH = 'data'
INPUT_FILENAME = 'master_data_final.csv'

INDEX = 'policy_number'
DATE_COLS = ['proposal_received_date', 'policy_issue_date', 'agent_dob', 'agent_doj']
NA_VALUES = ['', 'NA', 'N/A', 'NULL', 'null', '?', '*', '#N/A', '#VALUE!']
DTYPE_DICT = {'zipcode': 'str', 'agent_code': 'str'} ## THese columns should be string

In [5]:
def load_data(INPUT_FILEPATH, INPUT_FILENAME):
    input_df = pd.read_csv(os.path.join(INPUT_FILEPATH, INPUT_FILENAME),
                      index_col = INDEX,
                      na_values = NA_VALUES,
                      parse_dates = DATE_COLS,
                      dayfirst = True,
                      dtype = DTYPE_DICT)
                    
    return input_df

In [6]:
input_df = load_data(INPUT_FILEPATH, INPUT_FILENAME)

In [7]:
input_df.head(10)

,proposal_received_date,policy_issue_date,owner_age,owner_gender,marital_status,num_nominee,smoker,medical,education,occupation,experience,income,zipcode,county,state,negative_zipcode,family_member,existing_num_policy,has_critical_health_history,policy_term,payment_freq,annual_premium,sum_insured,agent_code,agent_dob,agent_doj,agent_status,agent_education,agent_age,agent_tenure_days,agent_persistency,last_6_month_submissions,average_premium,is_reinstated,prev_persistency,num_complaints,target_completion_perc,has_contacted_in_last_6_months,credit_score,lapse
policy_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2020-09-08,2020-09-08,22,Female,Single,1,No,No,Graduate,Sales,1,50000,19111,Philadelphia County,PA,0,1,0,0,25,Annually,377,580019,60503862,1997-07-03,2020-08-31,Active,Graduate,23,8,0.76,35,377,1,0.67,1,0.98,1,762,0
2,2020-08-24,2020-08-26,33,Female,Single,1,No,No,High School,Housewife,0,0,7764,Monmouth County,NJ,0,1,0,0,20,Quaterly,671,829002,60422713,1980-03-28,2018-05-17,Active,High School,40,830,0.87,54,671,0,0.00,4,0.90,1,794,0
3,2020-08-24,2020-08-24,29,Female,Married,1,No,No,Lt High School,Other Service,13,100000,26335,Braxton County,WV,0,2,1,0,10,Annually,210,659027,60450939,1978-06-25,2019-02-25,Inactive,Some College,42,546,0.93,0,579,0,0.00,8,0.93,1,800,1
4,2020-08-24,2020-08-24,57,Male,Divorced,1,No,Yes,Graduate,Military,36,242286,98241,Skagit County,WA,0,3,0,0,15,Annually,1863,612890,60394377,1993-05-05,2017-07-10,Active,Graduate,27,1141,0.78,29,1333,1,0.72,8,0.95,1,850,0
5,2020-08-24,2020-08-25,32,Female,Divorced,1,No,No,Post Graduate,Teacher,8,20434,78160,Wilson County,TX,0,1,1,0,20,Annually,730,879549,60491531,1967-12-05,2020-03-20,Active,Graduate,53,157,0.89,44,652,0,0.00,8,0.89,0,745,1
6,2020-08-24,2020-08-29,27,Male,Single,1,Yes,Yes,Graduate,Accountant,6,65000,15938,Cambria County,PA,0,1,0,0,20,Annually,941,862056,60408239,1986-06-05,2017-11-30,Active,Graduate,34,998,0.75,26,1890,0,0.00,8,0.83,0,720,0
7,2020-08-24,2020-08-24,39,Male,Married,1,No,No,High School,Govt Service,21,116539,28707,Jackson County,NC,0,3,0,1,10,Annually,331,529537,60476244,1984-11-03,2019-10-22,Active,Lt High School,36,307,0.81,45,926,0,0.00,13,0.92,0,686,0
8,2020-08-24,2020-09-30,39,Female,Widowed,1,Yes,Yes,Graduate,Shop Owner,18,56924,60129,DeKalb County,IL,0,1,0,1,15,Monthly,566,501114,60237645,NaT,2012-08-23,Active,High School,36,2923,0.98,60,1119,0,0.00,8,0.95,0,838,0
9,2020-08-24,2020-08-26,28,Male,Married,1,No,No,Graduate,IT Service,7,64762,33759,Pinellas County,FL,0,3,0,0,25,Annually,477,465743,60221480,1964-10-26,2011-10-13,Active,Lt High School,56,3238,0.70,40,842,0,0.00,27,0.88,1,773,1


#### Checking data info

In [8]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44948 entries, 1 to 44948
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   proposal_received_date          44948 non-null  datetime64[ns]
 1   policy_issue_date               44948 non-null  datetime64[ns]
 2   owner_age                       44948 non-null  int64         
 3   owner_gender                    44948 non-null  object        
 4   marital_status                  44948 non-null  object        
 5   num_nominee                     44948 non-null  int64         
 6   smoker                          44948 non-null  object        
 7   medical                         44948 non-null  object        
 8   education                       44948 non-null  object        
 9   occupation                      44948 non-null  object        
 10  experience                      44948 non-null  int64         
 11  in

In [9]:
input_df.columns

Index(['proposal_received_date', 'policy_issue_date', 'owner_age',
       'owner_gender', 'marital_status', 'num_nominee', 'smoker', 'medical',
       'education', 'occupation', 'experience', 'income', 'zipcode', 'county',
       'state', 'negative_zipcode', 'family_member', 'existing_num_policy',
       'has_critical_health_history', 'policy_term', 'payment_freq',
       'annual_premium', 'sum_insured', 'agent_code', 'agent_dob', 'agent_doj',
       'agent_status', 'agent_education', 'agent_age', 'agent_tenure_days',
       'agent_persistency', 'last_6_month_submissions', 'average_premium',
       'is_reinstated', 'prev_persistency', 'num_complaints',
       'target_completion_perc', 'has_contacted_in_last_6_months',
       'credit_score', 'lapse'],
      dtype='object')

#### Target variable distribution

In [10]:
input_df['lapse'].value_counts()/len(input_df)*100

0    58.338525
1    41.661475
Name: lapse, dtype: float64

#### Creating feature: time_to_issue

In [11]:
def create_time_to_issue(df) -> pd.DataFrame:
    df['time_to_issue'] = (df['policy_issue_date'] - df['proposal_received_date']).dt.days

    return df

In [12]:
input_df = create_time_to_issue(input_df)
input_df['time_to_issue'].describe()

count    44948.000000
mean         5.640451
std         11.153843
min          0.000000
25%          0.000000
50%          1.000000
75%          7.000000
max        494.000000
Name: time_to_issue, dtype: float64

#### Creating feature: prem_to_income_ratio

In [13]:
def create_prem_to_income_ratio(df) -> pd.DataFrame:
    df['prem_to_income_ratio'] = np.where(df['income'] == 0, 0, (df['annual_premium']/df['income']))

    return df

In [14]:
input_df = create_prem_to_income_ratio(input_df)
input_df['prem_to_income_ratio'].describe()

count    44948.000000
mean         0.016824
std          0.025457
min          0.000000
25%          0.004170
50%          0.008553
75%          0.017717
max          0.200000
Name: prem_to_income_ratio, dtype: float64

## --------------------------------------------------------------------------------------------------------------------------------

The objective of this exercise is to create a demoable solution (maybe not the best possible one, given the augmented data). Hence, we are not going to deep dive into EDA and hypothesis testing. Instead, we will focus on building the ML product using different technologies. 

## --------------------------------------------------------------------------------------------------------------------------------

#### Columns to remove

In [15]:
COLS_TO_REM = ['proposal_received_date','policy_issue_date', 'zipcode', 'county', 'state', 'agent_code', 'agent_dob', 'agent_doj']

- `proposal_received_date`: Derived `time_to_issue` from this column
- `policy_issue_date`: Derived `time_to_issue` from this column
- `zipcode`: Too many values, high cardinality
- `county`: Too many values, high cardinality
- `state`: Too many values, high cardinality
- `agent_code`: Id column
- `agent_dob`: Derived `agent_age` from this column
- `agent_doj`: Derived `agent_tenure_days` from this column

In [16]:
def filter_df(df, COLS_TO_REM) -> pd.DataFrame:
    df = df.drop(COLS_TO_REM, axis = 1)
    return df

In [18]:
temp_df = filter_df(input_df, COLS_TO_REM)
temp_df.shape

(44948, 34)

In [19]:
input_df.shape

(44948, 42)

## Feature Engineering Steps

#### Missing Value Imputation

We have only one column where missing value is present. `agent_persistency`. Impute missing value with median

In [20]:
agent_persistency_missing_perc = round(temp_df['agent_persistency'].isnull().mean()*100,2)

print(f'Total missing percentage of column agent_persistency is: {agent_persistency_missing_perc}% ')

Total missing percentage of column agent_persistency is: 0.26% 


In [21]:
MISSING_COL = ['agent_persistency']

#### One-Hot Encoding

In [22]:
ONE_HOT_COLS = ['owner_gender', 'marital_status', 'smoker', 'medical', 'education', 'occupation', 'payment_freq',  
                'agent_status', 'agent_education']

#### Normalisation

We will normalise the columns using `StandardScaler` because we have values at different scale

#### Creating train test split

In [23]:
temp_df.columns

Index(['owner_age', 'owner_gender', 'marital_status', 'num_nominee', 'smoker',
       'medical', 'education', 'occupation', 'experience', 'income',
       'negative_zipcode', 'family_member', 'existing_num_policy',
       'has_critical_health_history', 'policy_term', 'payment_freq',
       'annual_premium', 'sum_insured', 'agent_status', 'agent_education',
       'agent_age', 'agent_tenure_days', 'agent_persistency',
       'last_6_month_submissions', 'average_premium', 'is_reinstated',
       'prev_persistency', 'num_complaints', 'target_completion_perc',
       'has_contacted_in_last_6_months', 'credit_score', 'lapse',
       'time_to_issue', 'prem_to_income_ratio'],
      dtype='object')

In [24]:
FEATURES = ['owner_age', 'owner_gender', 'marital_status', 'num_nominee', 'smoker',
       'medical', 'education', 'occupation', 'experience', 'income',
       'negative_zipcode', 'family_member', 'existing_num_policy',
       'has_critical_health_history', 'policy_term', 'payment_freq',
       'annual_premium', 'sum_insured', 'agent_status', 'agent_education',
       'agent_age', 'agent_tenure_days', 'agent_persistency',
       'last_6_month_submissions', 'average_premium', 'is_reinstated',
       'prev_persistency', 'num_complaints', 'target_completion_perc',
       'has_contacted_in_last_6_months', 'credit_score',
       'time_to_issue', 'prem_to_income_ratio']

TARGET = 'lapse'

In [25]:
X_train, X_test, y_train, y_test = train_test_split(temp_df[FEATURES],
                                                    temp_df[TARGET],
                                                    test_size=0.3,
                                                    random_state = 786, 
                                                    shuffle = True,
                                                    stratify = temp_df[TARGET])

X_train.shape, X_test.shape

((31463, 33), (13485, 33))

#### Creating data transformation pipeline

In [26]:
model_input_pipe = Pipeline([
    
    ('imputer_num', mdi.MeanMedianImputer(imputation_method = 'median', variables = MISSING_COL )), 
    
    ('onehot_encoder', ce.OneHotEncoder(top_categories=None,
                                        variables= ONE_HOT_COLS,
                                        drop_last=True)),
    
    ('normalisation', StandardScaler())
    
    # ('clf', LogisticRegression(penalty,random_state = 786))
])

In [27]:
X_train_trf = model_input_pipe.fit_transform(X_train)

In [31]:
X_train_trf

array([[-0.13723333, -0.34844439,  0.15699084, ...,  1.76218543,
        -0.74815524, -0.58714994],
       [-0.22828715, -0.34844439,  0.24685535, ...,  1.76218543,
        -0.74815524, -0.58714994],
       [-0.41039479, -0.34844439, -0.38219622, ..., -0.56747717,
         1.33662099, -0.58714994],
       ...,
       [ 0.4090896 , -0.34844439,  0.87590693, ...,  1.76218543,
        -0.74815524, -0.58714994],
       [ 0.13592813, -0.34844439,  0.60631339, ..., -0.56747717,
        -0.74815524, -0.58714994],
       [-1.13882535, -0.34844439, -0.83151878, ..., -0.56747717,
         1.33662099, -0.58714994]])

#### Fitting the pipeline

In [32]:
with mlflow.start_run():

    mlflow.set_tag("developer", "tanmoy")

    penalty = 'l2'
    logreg = LogisticRegression(penalty= penalty)
    logreg.fit(X_train_trf, y_train)

    X_test_trf = model_input_pipe.transform(X_test)
    y_pred = logreg.predict(X_test_trf)

    round(accuracy_score(y_test, y_pred)*100,2)
    confusion_matrix(y_test, y_pred)
    classification_report(y_test, y_pred)